In [1]:
############## imports
# general
import statistics
import datetime
#from sklearn.externals import joblib # save and load models
import random
# data manipulation and exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

## machine learning stuff
# preprocessing
from sklearn import preprocessing
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_regression
# pipeline
from sklearn.pipeline import Pipeline
# train/testing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score  
# error calculations
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# models
from sklearn.linear_model import LinearRegression # linear regression
from sklearn.linear_model import BayesianRidge #bayesisan ridge regression
from sklearn.svm import SVC  # support vector machines classification
from sklearn.gaussian_process import GaussianProcessRegressor # import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsRegressor # k-nearest neightbors for regression
from sklearn.neural_network import MLPRegressor # neural network for regression
from sklearn.neural_network import MLPClassifier # neural network for classification
from sklearn.tree import DecisionTreeRegressor # decision tree regressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor  # random forest regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier # adaboost for classification
import xgboost as xgb
# saving models
# from sklearn.externals import joblib
import joblib
import matplotlib.pyplot as plt



In [2]:
import pandas as pd
import random
import math

# read in time point 1 for training
# train on it - model 1
# test on tp 2
# read tp 2
# add it to xgb - model 2
# test on tp 3
# read tp 3
# add it to xgb - model 3
# test on tp 4
# test model 1
data = pd.read_csv('data_no_2022_synthesized_nonstationary.csv')
data.head()

,yield,radiation,rain,avg_min_temp,avg_max_temp,class
0,0.766452,48010.950000,301.990000,12.436900,25.583800,0
1,0.768636,373381.000000,3837.770000,3.190400,14.261800,0
2,0.657273,328244.109091,3621.756667,4.053685,15.092736,0
3,0.912258,145586.280000,1621.280000,5.637700,16.831100,0
4,0.532857,159773.140000,1638.650000,5.033200,16.277100,0


In [3]:
print(len(data))

20090


In [4]:
data = data.rename(columns={'Yield (tons/acre)':'yield'})

In [5]:
xColumnsToKeep = ["radiation","rain", "avg_max_temp", "avg_min_temp"]

# the target to keep
yColumnsToKeep = ["yield"]

# get a dataframe containing the features and the targets
xDf = data[xColumnsToKeep]


yDf = data[yColumnsToKeep]


In [6]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, f_regression, SelectPercentile
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_error


In [7]:
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectPercentile

In [9]:
import optuna
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Define the number of features to select (e.g., you can set it as an integer or a percentage)
features = 4  # Example: Select top 3 features

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(xDf, yDf, test_size=0.2, random_state=42)

# Extract the first column from y_test and flatten it
y_test_flat = y_test.iloc[:, 0].values  # Use .iloc to select the first column and convert to 1D array

# Standardization
standardScaler = preprocessing.StandardScaler()

# Feature Selection
if type(features) == int:
    featureSelection = SelectKBest(f_regression, k=features)
elif type(features) == float:
    featuresPercentile = features / 100.0
    featureSelection = SelectPercentile(f_regression, percentile=featuresPercentile)
else:
    raise ValueError("The input 'features' is not an integer or a float. It should be.")

# Define the pipeline with standardization, feature selection, and the model
pipe = Pipeline(steps=[
    ('standardization', standardScaler),
    ('feature selection', featureSelection),
    ('model', RandomForestRegressor(random_state=42))
])

# Define the objective function for Optuna
def objective(trial):
    # Define the hyperparameter search space for RandomForestRegressor
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
    }
    
    # Update the pipeline model with the hyperparameters from Optuna trial
    pipe.set_params(model__n_estimators=param['n_estimators'],
                    model__max_depth=param['max_depth'],
                    model__min_samples_split=param['min_samples_split'],
                    model__min_samples_leaf=param['min_samples_leaf'],
                    model__max_features=param['max_features'],
                    model__bootstrap=param['bootstrap'])

    # Train the model on the training data
    pipe.fit(X_train, y_train)

    # Make predictions
    y_pred = pipe.predict(X_test)
    
    # Evaluation metrics
    mae = mean_absolute_error(y_test_flat, y_pred)
    r2 = r2_score(y_test_flat, y_pred)
    
    # Pearson's Correlation Coefficient (R)
    correlation_matrix = np.corrcoef(y_test_flat, y_pred)
    correlation = correlation_matrix[0, 1]
    
    # Symmetric Mean Absolute Percentage Error (SMAPE)
    def smape(y_true, y_pred):
        denominator = np.abs(y_true) + np.abs(y_pred)
        diff = np.abs(y_true - y_pred) / denominator
        diff[denominator == 0] = 0.0  # Handle the case where both true and predicted values are zero
        return 100 * np.mean(diff)
    
    smape_value = smape(y_test_flat, y_pred)
    
    # Composite objective: minimize MAE and SMAPE, maximize R² and correlation
    return mae

# Create a study object and optimize the objective function
study = optuna.create_study(direction='minimize')  # Minimize the objective
study.optimize(objective, n_trials=100)  # Perform 100 trials

# Get the best hyperparameters found by Optuna
best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

# Train the final model with the best hyperparameters
pipe.set_params(model__n_estimators=best_params['n_estimators'],
                model__max_depth=best_params['max_depth'],
                model__min_samples_split=best_params['min_samples_split'],
                model__min_samples_leaf=best_params['min_samples_leaf'],
                model__max_features=best_params['max_features'],
                model__bootstrap=best_params['bootstrap'])

pipe.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = pipe.predict(X_test)

# Print evaluation metrics
mae = mean_absolute_error(y_test_flat, y_pred)
r2 = r2_score(y_test_flat, y_pred)
correlation_matrix = np.corrcoef(y_test_flat, y_pred)
correlation = correlation_matrix[0, 1]
smape_value = smape(y_test_flat, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'R² (Coefficient of Determination): {r2}')
print(f'Pearson\'s Correlation Coefficient (R): {correlation}')
print(f'Symmetric Mean Absolute Percentage Error (SMAPE): {smape_value}')


[I 2025-04-20 18:55:49,027] A new study created in memory with name: no-name-3f6ad7b0-cc14-4401-b60f-d8da6817c91d
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
[I 2025-04-20 18:55:53,836] Trial 0 finished with value: 0.22744852693580184 and parameters: {'n_estimators': 142, 'max_depth': 14, 'min_samples_split': 6, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 0 with value: 0.22744852693580184.
/Libr

Best hyperparameters: {'n_estimators': 278, 'max_depth': 10, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False}
Mean Absolute Error (MAE): 0.22602891009780013
R² (Coefficient of Determination): 0.7118045639713848
Pearson's Correlation Coefficient (R): 0.8436979441830629
Symmetric Mean Absolute Percentage Error (SMAPE): 6.913350072011498


In [13]:
import statistics
import datetime
#from sklearn.externals import joblib # save and load models
import joblib
import random
# data manipulation and exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import statsmodels.api as sm
import math

## machine learning stuff
# preprocessing
from sklearn import preprocessing
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_regression
# pipeline
from sklearn.pipeline import Pipeline
# train/testing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sktime.utils.plotting import plot_series

# error calculations
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, f1_score, matthews_corrcoef
from sktime.performance_metrics.forecasting import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError
# models
from sklearn.linear_model import LinearRegression # linear regression
from sklearn.linear_model import BayesianRidge #bayesisan ridge regression
from sklearn.svm import SVR  # support vector machines regression
from sklearn.gaussian_process import GaussianProcessRegressor # import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsRegressor # k-nearest neightbors for regression
from sklearn.neural_network import MLPRegressor # neural network for regression
from sklearn.tree import DecisionTreeRegressor # decision tree regressor
from sklearn.ensemble import RandomForestRegressor  # random forest regression
from sklearn.ensemble import AdaBoostRegressor # adaboost for regression
import xgboost as xgb

skt_mape = MeanAbsolutePercentageError(symmetric=False)
skt_smape = MeanAbsolutePercentageError(symmetric=True)
skt_mae = MeanAbsoluteError()
skt_mse = MeanSquaredError()